In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np

from definitions import ROOT_DIR, main_activity_label_classes
from trainTestValSplit import getTrainTestValSplit
from preProcessing import preProcess
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.metrics import f1_score

In [2]:
train_concat, test_concat, val_concat = getTrainTestValSplit(
    _dataset_path=f'{ROOT_DIR}/dataset/ExtraSensory/ExtraSensory.per_uuid_features_labels'
)

In [3]:
train_x, train_y = preProcess(train_concat)
test_x, test_y = preProcess(test_concat)
val_x, val_y = preProcess(val_concat)

In [4]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)
val_x = val_x.fillna(0)

In [5]:
n_neurons: int = 1024
window_size: int = 1
epochs: int = 1
n_y: int = len(main_activity_label_classes)
batch_size: int = 256
n_features: int = len(train_x.columns)

In [6]:
train_x = train_x.to_numpy()
train_y = train_y.to_numpy()
test_x = test_x.to_numpy()
test_y = test_y.to_numpy()
val_x = val_x.to_numpy()
val_y = val_y.to_numpy()

In [7]:
# Source: https://stackoverflow.com/questions/43114460/is-there-a-way-to-reshape-an-array-that-does-not-maintain-the-original-size-or
def reshape_and_truncate(arr, shape):
    desired_size_factor = np.prod([n for n in shape if n != -1])
    if -1 in shape:  # implicit array size
        desired_size = arr.size // desired_size_factor * desired_size_factor
    else:
        desired_size = desired_size_factor
    return arr.flat[:desired_size].reshape(shape)

In [8]:
def encodeTarget(y):
    encoder = LabelEncoder()
    encoder.fit(y)
    y = encoder.transform(y)
    return np_utils.to_categorical(y)

In [9]:
train_y = encodeTarget(train_y)
test_y = encodeTarget(test_y)
val_y = encodeTarget(val_y)

In [10]:
train_x = reshape_and_truncate(train_x, (-1, window_size, n_features))
train_y = reshape_and_truncate(train_y, (-1, window_size, len(main_activity_label_classes)))

test_x = reshape_and_truncate(test_x, (-1, window_size, n_features))
test_y = reshape_and_truncate(test_y, (-1, window_size, len(main_activity_label_classes)))

val_x = reshape_and_truncate(val_x, (-1, window_size, n_features))
val_y = reshape_and_truncate(val_y, (-1, window_size, len(main_activity_label_classes)))

In [11]:
with tf.device('/gpu:0'):
    model = tf.keras.Sequential()

    layer = tf.keras.layers.Normalization(axis=-1)
    layer.adapt(train_x)
    model.add(layer)

    model.add(tfa.layers.ESN(n_neurons, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])))
    model.add(tf.keras.layers.Dense(n_y, activation=tf.keras.activations.softmax))

    model.summary()
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy())
    model.fit(x=train_x, y=train_y, epochs=epochs)#validation_data=(val_x, val_y), epochs=epochs)
    predict = model.predict(test_x)

Metal device set to: Apple M1 Pro


2022-06-22 15:14:05.818876: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-22 15:14:05.819018: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-06-22 15:14:06.044067: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-06-22 15:14:06.121640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-22 15:14:06.143627: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, None, 227)        455       
 n)                                                              
                                                                 
 esn (ESN)                   (None, None, 1024)        1282048   
                                                                 
 dense (Dense)               (None, None, 7)           7175      
                                                                 
Total params: 1,289,678
Trainable params: 7,175
Non-trainable params: 1,282,503
_________________________________________________________________


2022-06-22 15:14:50.084169: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8322/8322 [==============================] - 81s 10ms/step - loss: 0.5972


2022-06-22 15:16:11.454504: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2466/2466 [==============================] - 13s 5ms/step


In [14]:
len([_ for _ in predict if _[0] == 1]) / len(predict)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()